In [2]:
import numpy as np
import pandas as pd
import displacement_solver
import constitutive
import mesh_gen
import quadrature
import stress_gauss
import patch_n_int_nodes
import gauss_pt_coord
import stress_nodes_dc
import spr_stress

from sklearn.preprocessing import MinMaxScaler 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.models import model_from_json

2023-04-28 02:36:31.722741: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-28 02:36:31.732074: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-28 02:36:31.771259: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-28 02:36:31.772065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 02:36:37.969871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
# ............................Inputs Parameters......................#

#Doamin geometry
domain_coord = np.array([[2, 0], [20, 0], [20, 10], [10, 0], [0, 2]]);

# Body force components
b = np.array([[0], [0]])

# Traction Components
q = 1/8      #unirt force for unit length

T = np.array([[q, 0]]);

# Young's modulus
E = 1.0

# Poission's ration
nu = 1/3;

# problem type (0--->plane stress, 1---->plane strain)
problem_type = 0;

#Element type used for meshing (0---->4 nodes quadrilateral)
el_type = 0;

# No. of Gauss points required forintegration
ngp2d = 1;
ngp1d = 2;

# Number of mesh in one direction.
N = 5;

In [4]:
u = displacement_solver.solve_fem_plat_with_hole(N, E, nu, ngp2d, ngp1d, el_type, problem_type, domain_coord, b, T)
# print(u_node)

In [5]:
# ms = mesh size ----> Number of element in one direction
ms = N;

#total number of elements
nel = 2*ms*ms 

# reshaping u into u_nodes with displacement in x-direction in first column and y-direciton in second column
u_nodes = u.reshape((2*(ms+1)*(ms+1)-(ms+1), 2))
# print(u_nodes)

In [6]:
# constitutive relation matrix, calculated using the fuction "Constitutube" with input E->(Young's Modulus), nu(Poisson's raton), problem_type(plane stress or plane strain)
C = constitutive.constitutive(E, nu, problem_type)
# print(C);

In [7]:
# mesh generation 
nx = ms #number of element in x-direction
ny = ms #number of element in y-direction

# el_type = element type specifiedc. 0----> q4, q-----> q8, 2-----> q9
el_type = 0;

#mesh_obj = object created to calculated nodal coordinates ans connectivity array using functon "connectivity" and "coord_array" 
#input nx->number of element in x-direction, xy->number of element in y-direction, domain_coord->coordinates of the corner points and mid-points of the cook's skew beam problem, el_type->element type specidied.

mesh_obj = mesh_gen.platWithHole(nx, ny, domain_coord.reshape(10, 1), el_type);
connect = mesh_obj.connectivity();
# print(connect)
coord = mesh_obj.coord_array();

In [8]:
strs_ref = pd.read_csv('stress/spr/spr_ms_18_to_143.csv')
# strs_ref = strs_ref.abs()

In [10]:
# print(strs_ref)

        Unnamed: 0         x         y  sigma_x_spr  sigma_y_spr  sigma_xy_spr
0                0   2.00000  0.000000    -1.407433    -1.794387      0.541425
1                1   3.00000  0.000000    -1.039010     0.379486      0.030181
2                2   4.00000  0.000000     0.706419     0.009702     -0.311681
3                3   5.00000  0.000000     1.405650    -0.383432     -0.168296
4                4   6.00000  0.000000     0.187994     0.278646      0.162772
...            ...       ...       ...          ...          ...           ...
131214      131214   9.72028  0.055944    -1.604075     0.295498     -3.185185
131215      131215   9.79021  0.041958    -1.286208     0.185271     -2.206302
131216      131216   9.86014  0.027972    -0.941738     0.191465     -1.249732
131217      131217   9.93007  0.013986    -1.734138     0.105668     -1.496688
131218      131218  10.00000  0.000000    -4.111336    -0.151720     -2.237556

[131219 rows x 6 columns]


In [11]:
var = 'x_y_xy'

In [15]:
X = strs_ref[['x', 'y']];
# X.to_csv('Coord/mesh_18-143'+'.csv');
Y = strs_ref[['sigma_x_spr', 'sigma_y_spr', 'sigma_xy_spr']];

In [13]:
# X = pd.read_csv('Coord/mesh_256.csv')
# X = X.loc[:, ~X.columns.str.contains('Unnamed')];
# y = strs_ref;
# y = y.loc[:, ~y.columns.str.contains('Unnamed')];
# # y = y.loc[:, ~y.columns.str.contains('x')];
# y = y.drop(columns=['x', 'y'])
# # y = y.loc[:, ~y.columns.str.contains('y')];

In [18]:
# print(X.head())
# print(Y.head())

In [19]:
scaler = MinMaxScaler()
scaler.fit(X)
scaled_x = scaler.transform(X)
scaled_test = scaler.transform(coord)

/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [158]:
# print(X.head());
# print(scaled_x);
# print(scaled_test);
# print(len(scaled_x));
# print(len(scaled_test));

In [23]:
#ANN one model in whole domain
model1= Sequential()
model1.add(Dense(units=50, activation='relu'))
model1.add(Dense(units=44, activation='relu'))
model1.add(Dense(units=38, activation='relu'))
model1.add(Dense(units=31, activation='relu'))
model1.add(Dense(units=25, activation='relu'))
model1.add(Dense(units=18, activation='relu'))
model1.add(Dense(units=12, activation='relu'))
model1.add(Dense(units=5, activation='relu'))
model1.add(Dense(units=3,activation='linear'))
model1.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))



In [24]:
 es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',  mode = 'min', verbose = 1, patience = 50);
 np.random.seed(1)
 history = model1.fit(x = scaled_x, y = Y, batch_size=256, epochs=300, verbose = 1, validation_split = 0.1, callbacks= [es]);

Epoch 1/300
462/462 [==============================] - 13s 18ms/step - loss: 8824992783400960.0000 - val_loss: 270866592038912.0000
Epoch 2/300
462/462 [==============================] - 7s 15ms/step - loss: 8824996541497344.0000 - val_loss: 270866592038912.0000
Epoch 3/300
462/462 [==============================] - 7s 15ms/step - loss: 8824997615239168.0000 - val_loss: 270866592038912.0000
Epoch 4/300
462/462 [==============================] - 8s 16ms/step - loss: 8824992246530048.0000 - val_loss: 270866592038912.0000
Epoch 5/300
462/462 [==============================] - 8s 18ms/step - loss: 8824992783400960.0000 - val_loss: 270866592038912.0000
Epoch 6/300
462/462 [==============================] - 7s 16ms/step - loss: 8824996541497344.0000 - val_loss: 270866592038912.0000
Epoch 7/300
462/462 [==============================] - 7s 15ms/step - loss: 8824990635917312.0000 - val_loss: 270866592038912.0000
Epoch 8/300
462/462 [==============================] - 7s 16ms/step - loss: 882499

In [25]:
var = 'x_y_xy'

In [26]:
loss = pd.DataFrame(model1.history.history);

In [28]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('train_loss', fontsize=15)
# plt.ylabel('Loss', fontsize=15)
# plt.xlabel('Epoch', fontsize=15)
# plt.legend(['train', 'val'], loc='upper right')
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.show()

In [29]:
#Saving train model with different optimizer.
model_json = model1.to_json()
with open('Model/model3_'+var+'.json', 'w') as json_file:
    json_file.write(model_json)

#serialize weights to HDF5 file
model1.save_weights('Model/model3_'+var+'.h5')


In [31]:
# # load json and create model
json_file = open('Model/model3_x_y_xy.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json);

# #load weights into new model
loaded_model.load_weights('Model/model3_x_y_xy.h5') 

In [32]:
#Stress calculation at Gauss points
stress = np.zeros((nel, ngp2d*ngp2d, 3))
strains = np.zeros((nel, ngp2d*ngp2d, 3))

for i in range(nel):
    stress_i_g = np.zeros((ngp2d*ngp2d, 3))
    strains_i_g = np.zeros((ngp2d*ngp2d, 3))

    stress_i_g, strains_i_g = stress_gauss.get_element_stress(
        i, ngp2d, el_type, connect, coord, u, C)

    stress[i][:][:] = stress_i_g
    strains[i][:][:] = strains_i_g.reshape((1, 3))


In [34]:
# calcuation of gauss coordinates.
gauss_coords = np.zeros((nel, ngp2d*ngp2d, 2))
gp, weights = quadrature.quadrature(ngp2d)
for i in range(nel):
    node = connect[i, :]
    vertex_coord = coord[node, :].reshape(-1)
    gauss_coords[i][:][:] = gauss_pt_coord.gauss_pts(
        ngp2d, vertex_coord, gp, el_type)
gauss_coords = gauss_coords.reshape(gauss_coords.shape[0], -1)


In [35]:
#creation of patches for spr_stress;
patch, n_patches, int_nodes = patch_n_int_nodes.patch_n_int_nodes_plat_with_hole(ms)


In [36]:
#spr_stress STRESS Calculations
stress_spr = spr_stress.spr_plat_with_hole(gauss_coords, coord, connect, stress,
                     int_nodes, n_patches, patch, ms)
# stress_spr


In [37]:
#Directly calculated stress
stress_dc, strain_dc = stress_nodes_dc.stress_dc(
    connect, coord, u, nel, el_type, C);


In [39]:
#outter points in
sp = []
# print(ms)
# ms = 4
temp = ms+1
for i in range(2*ms-1):
    sp.append(temp)
    sp.append(temp+ms)
    temp = temp+ms+1
for i in range(ms+1):
    sp.append(temp)
    temp = temp+1

for i in range(ms+1):
    sp.append(i)

sp = sorted(sp)
# print(sp);

In [40]:
coord_corner = coord[sp]
# print(coord_corner)

In [41]:
scaled_test_corner = scaler.transform(coord_corner)
# scaled_test_corner

/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [42]:
ref_stress = pd.read_csv('ref_stress/ref_stress_for_ms_5_from_320.csv');
ref_stress = ref_stress.loc[:, ~ref_stress.columns.str.contains('^Unnamed')];
ref_stress = ref_stress.drop(columns=['x', 'y'])
# print(ref_stress)

ref_stress = ref_stress.to_numpy();
# ref_stress

In [43]:
a = ref_stress.T
# print(len(a));

In [44]:
b = stress_spr.T
# print(len(b))

In [45]:
err_spr_abs = np.abs((a-b));
err_spr_perc = np.abs((a-b)/a);
err_dc_abs = np.abs((a-stress_dc.T));
err_dc_perc = np.abs((a-stress_dc.T)/a);
# err_spr_abs
# err_dc_abs

In [46]:
test = pd.DataFrame(scaled_test, columns =['a', 'b']);
# test

In [47]:
c = loaded_model.predict(test);
# len(c)

3/3 [==============================] - 0s 7ms/step


In [48]:
err_ann_abs = np.abs((a-c.T))
err_ann_perc = np.abs((a-c.T)/a)
# err_ann_abs

In [49]:

alll = np.concatenate([coord, 
                        a.reshape(a.T.shape[0], 3), 
                        b.reshape(b.T.shape[0], 3), 
                        c.reshape(c.shape[0], 3), 
                        err_spr_abs.reshape(err_spr_abs.T.shape[0], 3), 
                        err_ann_abs.reshape(err_ann_abs.T.shape[0],3), 
                        err_dc_abs.reshape(err_dc_abs.T.shape[0],3),
                        err_spr_perc.reshape(err_spr_perc.T.shape[0],3),
                        err_ann_perc.reshape(err_ann_perc.T.shape[0],3),
                        err_dc_perc.reshape(err_dc_perc.T.shape[0],3) 
                        ], 
                        axis = 1);
# alll

In [50]:
tdf =pd.DataFrame(alll, columns=['x', 
                                'y', 
                                'ref_stress_x',
                                'ref_stress_y',
                                'ref_stress_xy', 
                                'spr_stress_x',
                                'spr_stress_y',
                                'spr_stress_xy', 
                                'ann_stress_x',
                                'ann_stress_y',
                                'ann_stress_xy', 
                                'err_spr_abs_x',
                                'err_spr_abs_y',
                                'err_spr_abs_xy', 
                                'err_ann_abs_x',
                                'err_ann_abs_y',
                                'err_ann_abs_xy', 
                                'err_dc_abs_x',
                                'err_dc_abs_y',
                                'err_dc_abs_xy',
                                'err_spr_perc_x',
                                'err_spr_perc_y',
                                'err_spr_perc_xy', 
                                'err_ann_perc_x',
                                'err_ann_perc_y',
                                'err_ann_perc_xy', 
                                'err_dc_perc_x',
                                'err_dc_perc_y',
                                'err_dc_perc_xy'
                                ]);
# print(tdf);

In [51]:
tdf.to_csv('Results/all_matlab_train_model3_ref_'+var+'.csv');
tdf.head()

,x,y,ref_stress_x,ref_stress_y,ref_stress_xy,spr_stress_x,spr_stress_y,spr_stress_xy,ann_stress_x,ann_stress_y,...,err_dc_abs_xy,err_spr_perc_x,err_spr_perc_y,err_spr_perc_xy,err_ann_perc_x,err_ann_perc_y,err_ann_perc_xy,err_dc_perc_x,err_dc_perc_y,err_dc_perc_xy
0,2.0,0.0,-0.000173,0.144666,-9.529965,-0.977762,-0.694206,-0.216256,-0.008072,-0.158129,...,10.428038,5663.061093,5.798695,0.977308,45.761455,1.055799,0.999153,61341.118638,27.907634,1.094237
1,5.6,0.0,7.839685,2.178640,2.840977,-0.072839,-0.037187,0.069772,-0.008072,-0.158129,...,3.660670,1.009291,1.017069,0.975441,1.001030,1.003705,1.002841,1.164220,0.844558,1.288525
2,9.2,0.0,-0.000071,-91.354094,-251.462422,-0.687458,-1.050563,-0.762537,-0.008072,-0.158129,...,249.095729,9701.518261,0.988500,0.996968,112.928299,0.999912,0.999968,173905.116993,1.021424,0.990588
3,12.8,0.0,13.157520,5.933357,1.333309,-0.303101,-0.069180,0.015766,-0.008072,-0.158129,...,0.261244,1.023036,1.011660,0.988175,1.000614,1.001360,1.006054,0.955942,1.091159,0.195937
4,16.4,0.0,0.000929,-3946.754606,-10292.325574,-0.482481,-1.214000,-0.727692,-0.008072,-0.158129,...,10293.383032,520.619879,0.999692,0.999929,9.693574,0.999998,0.999999,9889.297830,0.998992,1.000103
